# Quaternions representing rotations SO(3)
This notebook is a short sumamry of how SO(3) is handled in Pinocchio, and mostly quaternion representation. 
## SO(3), Euler angles, rotations

In [71]:
import magic_donotload

In [72]:
import pinocchio
from pinocchio import SE3, Quaternion
import numpy as np
from numpy.linalg import norm
M = SE3.Random()
R = M.rotation
p = M.translation
print('R =\n' + str(R))
print('p =\n' + str(p.T))

R =
[[-0.65353882  0.30654233  0.6920396 ]
 [-0.60676219  0.33439086 -0.72112578]
 [-0.4524673  -0.89118716 -0.03253912]]
p =
[ 0.75655539 -0.31149808  0.62981733]


A rotation is simply a 3x3 matrix. It has a unit norm:

In [73]:
print(R @ R.T)

[[ 1.00000000e+00 -2.22570716e-17 -1.25003110e-16]
 [-2.22570716e-17  1.00000000e+00  1.18750834e-16]
 [-1.25003110e-16  1.18750834e-16  1.00000000e+00]]


It can be equivalently represented by a quaternion. Here we have made the choice to create a specific class for quaternions (i.e. they are not vectors, and can be e.g. multiplied), but you can get the 4 coefficients with the adequate method. Note that the corresponding vector is also of norm 1.

In [74]:
quat = Quaternion(R)
print(norm(quat.coeffs()))

1.0


Angle-axis representation are also implemented in the class AngleAxis.

In [75]:
from pinocchio import AngleAxis
utheta = AngleAxis(quat)
print(utheta.angle, utheta.axis)

2.3129049136383455 [-0.11536701  0.77641576 -0.61957164]


You can display rotation in the viewer.

In [76]:
from utils.meshcat_viewer_wrapper import MeshcatVisualizer
viz = MeshcatVisualizer()
viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/


In [78]:
viz.addBox('world/box', [.1, .2, .3], [1, 0, 0, 1])
viz.applyConfiguration('world/box', [.1, .2, .3] + list(quat.coeffs()))

### Quaternion you said?
Quaternions are "complex of complex", introduced form complex as complex are from reals. Let's try to understand what they contains in practice. Quaternions are of the form w+xi+yj+zk, with w,x,y,z real values, and i,j,k the 3 imaginary numbers. We store them as 4-d vectors, with the real part first: quat = [x,y,z,w]. We can interprete w as encoding the cosinus of the rotation angle. Let's see that.

In [79]:
from numpy import arccos
print(arccos(quat[3]))
print(AngleAxis(quat).angle)

1.1564524568191727
2.3129049136383455


Indeed, w = cos(θ/2). Why divided by two? For that, let's see how the quaternion can be used to represent a rotation. We can encode a 3D vector in the imaginary part of a quaternion.

In [80]:
p = np.random.rand(3)
qp = Quaternion(0., p[0], p[1], p[2])
print(qp)

(x,y,z,w) =  0.666993  0.263903 0.0502224         0



The real product extends over quaternions, so let's try to multiply quat with p:

In [81]:
print(quat * qp)

(x,y,z,w) =   0.453889  -0.266732  -0.481693 -0.0886394



Well that's not a pure imaginary quaternion anymore. And the imaginary part does not contains somethig that looks like the rotated point:

In [82]:
print(quat.matrix() @ p)

[-0.32025213 -0.35267576 -0.53861379]


The pure quaternion is obtained by multiplying again on the left by the conjugate (w,-x,-y,-z).

In [83]:
print(quat * qp * quat.conjugate())

(x,y,z,w) =    -0.320252    -0.352676    -0.538614 -2.77556e-17



That is a pure quaternion, hence encoding a point, and does corresponds to R*p. Magic, is it not? We can prove that the double product of quaternion does corresponds to the rotation. Indeed, a quaternion rather encode an action (a rotation) in $R^4$, but which moves our point p outside of $R^3$. The conjugate rotation brings it back in $R^3$ but applies a second rotation. Since we rotate twice, it is necessary to apply only half of the angle each time.
What if we try to apply the rotation quat on the imaginary part of the quaternion?

In [84]:
qim = Quaternion(quat)  # copy
qim[3] = 0
print(qim, quat * qim * quat.conjugate())

(x,y,z,w) = -0.105605  0.710716 -0.567144         0
 (x,y,z,w) =   -0.105605    0.710716   -0.567144 2.77556e-17



What kind of conclusion can we get from this? What geometrical interpretation can we give to $q_{im}$? What about $||q_{im}||$?

### The SLERP example
Let's practice! Implement a linear interpolation between two position p0 and p1, i.e. find the position p(t) with t varying from 0 to 1, with p(0)=p0, p(1)=p1 and continuity between the two extrema.

In [87]:
%do_not_load appendix/solution_lerp.py

LERP with quaternions is not working because they are not normalized. Instead we can take either the normalization of the LERP (NLERP), or the spherical LERP (SLERP). 

In [92]:
%do_not_load appendix/solution_slerp.py

If you compare the animations for slerp and nlerp  (possibly by running your code several times), you will see that sometimes the rotation is not similar in both cases. Contrary to our lerp implementation, slerp is guaranteed to interpolate following the shortest path, while sometimes lerp will take the "long way home", resulting in a longer rotation (which as a result is animated faster in the solution code)

## Cheat sheet

You can convert quaternion to rotation matrix and create SE3 objects as follows:


In [29]:
qu = Quaternion(.7,.2,.2,.6)# Quaternion: take care that norm <= 1 (and approx 1)
R  = qu.matrix()                # Create a rotation matrix from quaternion
p  = np.array([0.,0.,0.77])     # Translation (R3) vector)
M  = SE3(R,p)                   # Create a nomogeneous matrix from R,P

# Typical tool position
from pinocchio.utils import rotate
M = SE3(rotate('z',1.)@rotate('x',.2), np.array([0.1,0.02,.65]))